<a href="https://colab.research.google.com/github/abriyanyusuf/C23PS423_ML/blob/main/Inference_Testing_Model_and_mAP_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.&nbsp;Install TensorFlow Object Detection Dependencies

Same with the main notebook for Training Object Detection, we need to install all dependencies

In [ ]:
# Clone the tensorflow models repository from GitHub
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3909, done.
remote: Counting objects: 100% (3909/3909), done.
remote: Compressing objects: 100% (3025/3025), done.
remote: Total 3909 (delta 1127), reused 1980 (delta 831), pack-reused 0
Receiving objects: 100% (3909/3909), 49.65 MiB | 13.29 MiB/s, done.
Resolving deltas: 100% (1127/1127), done.


In [ ]:
# Copy setup files into models/research folder
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
#cp object_detection/packages/tf2/setup.py .

In [ ]:
# Modify setup.py file to install the tf-models-official repository targeted at TF v2.8.0
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
# Install the Object Detection API
!pip install /content/models/research/

# Need to downgrade to TF v2.8.0 due to Colab compatibility bug with TF v2.10 (as of 10/03/22)
!pip install tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.6/116.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing m

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 40.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.1
    Uninstalling tensorflow-2.8.1:
      Successfully uninstalled tensorflow-2.8.1


In [ ]:
# Run Model Bulider Test file, just to verify everything's working properly
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl5mutexC1Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFN3tsl8StatusOrISt10u

#2. Upload Image Dataset and Prepare Training Data

##2.1. Mounting Google Drive and Copy zip file

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


We will copy our dataset from gdrive

In [ ]:
!cp '/content/gdrive/MyDrive/Capstone Project Machine Learning/ECUSTFD.zip' '/content'

##2.2. Split Images Into Train, Validation, and Test Folders OPTIONAL

We will create directory for train, test, and validation and unzip the dataset and put the result to desired directory. These steps is optional if we forget the `images` folder in Training Object Detection before

In [ ]:
#Run this code if we faced error, it will remove all the changes and start from the beginning
#%cd /content

In [ ]:
#!rm -rf images

Start create directory 

In [ ]:
%cd ..

/


In [ ]:
!mkdir /content/images
!mkdir /content/images/train; mkdir /content/images/validation; mkdir /content/images/test

In [ ]:
%cd /content

/content


In [ ]:
!unzip -q ECUSTFD.zip -d /content/images/all

We will move 80% of the images to the tran folder, 10% to validation and 10% to the test folder. In this code we will use several library \: 

1. The Python Glob module searches all path names looking for files matching a specified pattern according to the rules dictated by the Unix shell.

2. Pathlib is a native Python library for handling files and paths on your operating system. It offers a bunch of path methods and attributes that make handling files more convenient than using the os module.

3. The random module in Python defines a series of functions for generating or manipulating random integers. The import random loads the random module, which contains a number of random number generation-related functions.

4. Python os system function allows us to run a command in the Python script



```
import glob
from pathlib import Path
import random
import os


# Define paths to image folders
image_path = '/content/images/all/ECUSTFD/ImagesWithXml'
train_path = '/content/images/train'
val_path = '/content/images/validation'
test_path = '/content/images/test'

# Get list of all images
jpg_file_list = [path for path in Path(image_path).rglob('*.jpg')]
JPG_file_list = [path for path in Path(image_path).rglob('*.JPG')]
png_file_list = [path for path in Path(image_path).rglob('*.png')]
bmp_file_list = [path for path in Path(image_path).rglob('*.bmp')]

file_list = jpg_file_list + JPG_file_list + png_file_list + bmp_file_list
file_num = len(file_list)
print('Total images: %d' % file_num)

# Determine number of files to move to each folder
train_percent = 0.8  # 80% of the files go to train
val_percent = 0.1    # 10% go to validation
test_percent = 0.1   # 10% go to test
train_num = int(file_num*train_percent)
val_num = int(file_num*val_percent)
test_num = file_num - train_num - val_num
print('Images moving to train: %d' % train_num)
print('Images moving to validation: %d' % val_num)
print('Images moving to test: %d' % test_num)

# Select 80% of files randomly and move them to train folder
for i in range(train_num):
    move_me = random.choice(file_list) #Choose randomly one file from file list
    fn = move_me.name #store file name in fn
    base_fn = move_me.stem #store file name without extension
    parent_path = move_me.parent #get root path from move_me
    xml_fn = base_fn + '.xml' #add extension xml
    os.rename(move_me, train_path+'/'+fn) #move file move_me to train folder
    #move xml from choosen file to train folder
    os.rename(os.path.join(parent_path,xml_fn),os.path.join(train_path,xml_fn))
    file_list.remove(move_me) #remove choosen file from file list to avoid it will choosen again

# Select 10% of remaining files and move them to validation folder
for i in range(val_num):
    move_me = random.choice(file_list)
    fn = move_me.name
    base_fn = move_me.stem
    parent_path = move_me.parent
    xml_fn = base_fn + '.xml'
    os.rename(move_me, val_path+'/'+fn)
    os.rename(os.path.join(parent_path,xml_fn),os.path.join(val_path,xml_fn))
    file_list.remove(move_me)

# Move remaining files to test folder
for i in range(test_num):
    move_me = random.choice(file_list)
    fn = move_me.name
    base_fn = move_me.stem
    parent_path = move_me.parent
    xml_fn = base_fn + '.xml'
    os.rename(move_me, test_path+'/'+fn)
    os.rename(os.path.join(parent_path,xml_fn),os.path.join(test_path,xml_fn))
    file_list.remove(move_me)
```



We already create .py contain code above in our repo. So we will use it by using this code below

In [ ]:
!wget https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/Split_to_TrainValTest.py

--2023-06-10 09:19:39--  https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/Split_to_TrainValTest.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2591 (2.5K) [text/plain]
Saving to: ‘Split_to_TrainValTest.py’

Split_to_TrainValTe 100%[===================>]   2.53K  --.-KB/s    in 0s      

2023-06-10 09:19:39 (52.4 MB/s) - ‘Split_to_TrainValTest.py’ saved [2591/2591]



We will run the code to split our data

In [ ]:
!python Split_to_TrainValTest.py

Total images: 2978
Images moving to train: 2382
Images moving to validation: 297
Images moving to test: 299


##2.3 Copy images.zip from GDrive and Unzip
If we already download images.zip that contain train, validation, and test data from the previous notebook. We can just use it for this session

In [ ]:
!cp '/content/gdrive/MyDrive/Capstone Project Machine Learning/images.zip' '/content'

In [ ]:
%cd /content
!unzip images.zip

##2.4 Create Labelmap and TFRecords

We need to create labelmap for the detector and convert the images into a data file format called TFRecords, which are used by TensorFlow for training. 
What we need to do?
1. Defining a label map for our classes by creating a "labelmap.txt"
2. Convert the data into TFRecord format

In [ ]:
##Creates labelmap.txt that contain list of label below
%%bash
cat <<EOF >> /content/labelmap.txt #command to append text inside txt file
banana 
apple 
bread 
bun 
doughnut 
egg 
fired_dough_twist 
grape 
lemon 
litchi 
mango 
mooncake 
orange 
peach 
pear 
plum 
qiwi 
sachima 
tomato
coin 
EOF



We will use data conversion scripts from the GitHub. By running this code, it will create TFRecord files for the train and validation datasets, as well as a labelmap.pbtxt file which contains the label map in a different format.

In [ ]:
# Download data conversion scripts
!wget https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/Create_CSV_from_VOC.py
!wget https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/CSV_to_TFRecord_Converter.py

--2023-06-10 09:19:40--  https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/Create_CSV_from_VOC.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1370 (1.3K) [text/plain]
Saving to: ‘Create_CSV_from_VOC.py’

Create_CSV_from_VOC 100%[===================>]   1.34K  --.-KB/s    in 0s      

2023-06-10 09:19:40 (83.2 MB/s) - ‘Create_CSV_from_VOC.py’ saved [1370/1370]

--2023-06-10 09:19:40--  https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/CSV_to_TFRecord_Converter.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 

In [ ]:
# Create CSV data files and TFRecord files
!python3 Create_CSV_from_VOC.py


Successfully converted xml to csv.
Successfully converted xml to csv.


In [ ]:
!python3 CSV_to_TFRecord_Converter.py --csv_input=images/train_labels.csv --labelmap=labelmap.txt --image_dir=images/train --output_path=train.tfrecord
!python3 CSV_to_TFRecord_Converter.py --csv_input=images/validation_labels.csv --labelmap=labelmap.txt --image_dir=images/validation --output_path=val.tfrecord

Successfully created the TFRecords: /content/train.tfrecord
Successfully created the TFRecords: /content/val.tfrecord


We will store the locations of the TFRecord and labelmap files as variables so we can reference them later in this Colab session

In [ ]:
train_record_fname = '/content/train.tfrecord'
val_record_fname = '/content/val.tfrecord'
label_map_pbtxt_fname = '/content/labelmap.pbtxt'

#Upload our model and Unzip

In [ ]:
%cd /content
!mkdir '/content/quant model'
!mkdir '/content/standard model'

/content


In [ ]:
!cp '/content/gdrive/MyDrive/Capstone Project Machine Learning/custom_model_lite.zip' '/content/standard model'

In [ ]:
%cd '/content/standard model'
!unzip custom_model_lite.zip

/content/standard model
Archive:  custom_model_lite.zip
   creating: custom_model_lite/
  inflating: custom_model_lite/labelmap.pbtxt  
  inflating: custom_model_lite/pipeline_file.config  
  inflating: custom_model_lite/labelmap.txt  
  inflating: custom_model_lite/detect.tflite  
   creating: custom_model_lite/saved_model/
  inflating: custom_model_lite/saved_model/saved_model.pb  
   creating: custom_model_lite/saved_model/assets/
   creating: custom_model_lite/saved_model/variables/
  inflating: custom_model_lite/saved_model/variables/variables.index  
  inflating: custom_model_lite/saved_model/variables/variables.data-00000-of-00001  


In [ ]:
!cp '/content/gdrive/MyDrive/Capstone Project Machine Learning/custom_model_lite_quant.zip' '/content/quant model'
%cd '/content/quant model'
!unzip custom_model_lite_quant.zip


/content/quant model
Archive:  custom_model_lite_quant.zip
   creating: custom_model_lite/
  inflating: custom_model_lite/labelmap.pbtxt  
  inflating: custom_model_lite/detect_quant.tflite  
  inflating: custom_model_lite/pipeline_file.config  
  inflating: custom_model_lite/labelmap.txt  
  inflating: custom_model_lite/detect.tflite  
   creating: custom_model_lite/saved_model/
  inflating: custom_model_lite/saved_model/saved_model.pb  
   creating: custom_model_lite/saved_model/assets/
   creating: custom_model_lite/saved_model/variables/
  inflating: custom_model_lite/saved_model/variables/variables.index  
  inflating: custom_model_lite/saved_model/variables/variables.data-00000-of-00001  


# 3.&nbsp;Test TensorFlow Lite Model and Calculate mAP for Standard Model


### 3.1 Inference test images
We will test our model using images inside test folder. We will test how well the model works before we download the model.

The code is an implementation of a function called tflite_detect_images. This function is used to perform inference with the TFLite model and display the results of object detection in images.

`os`: This package provides functions to interact with the operating system, such as reading directories, setting environment variables, etc.

`cv2` (OpenCV): This package is used to manipulate and process images and videos. In this context, it is used to read and manipulate images.

`numpy`: This package provides support for multidimensional array and matrix operations, as well as various efficient math functions. Used in this context to manipulate processed image data.

`sys`: This package provides access to several variables and functions that are widely used to manipulate various parts of the Python runtime environment.

`glob`: This package is used to match files and directories using certain patterns. In this context, it is used to get a list of image files in a folder that matches the specified pattern.

`random`: This package provides functions to generate random numbers. In this context, it is used to select a random number of images from a list of image files.

`importlib.util`: This package provides functions related to importing dynamic modules. In this context, it is used to load the TFLite model into memory.

`tensorflow.lite.python.interpreter`: This package provides the Interpreter for running TensorFlow Lite models. It is used in this context to load the TFLite model into memory and run object inference on images.

`matplotlib`: This package provides functions for creating graph and plot visualizations. It is used in this context to display images of object detection results.

In [ ]:
# Import packages
import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

### Define function for inferencing with TFLite model and displaying results

def tflite_detect_images(modelpath, imgpath, lblpath, min_conf=0.5, num_test_images=10, savepath='/content/results', txt_only=False):

  # Grab filenames of all images in test folder
  images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

  # Load the label map into memory
  with open(lblpath, 'r') as f:
      labels = [line.strip() for line in f.readlines()]

  # Load the Tensorflow Lite model into memory
  interpreter = Interpreter(model_path=modelpath)
  interpreter.allocate_tensors()

  # Get model details
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()
  height = input_details[0]['shape'][1]
  width = input_details[0]['shape'][2]

  float_input = (input_details[0]['dtype'] == np.float32)

  input_mean = 127.5
  input_std = 127.5

  # Randomly select test images
  images_to_test = random.sample(images, num_test_images)

  # Loop over every image and perform detection
  for image_path in images_to_test:

      # Load image and resize to expected shape [1xHxWx3]
      image = cv2.imread(image_path)
      image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      imH, imW, _ = image.shape 
      image_resized = cv2.resize(image_rgb, (width, height))
      input_data = np.expand_dims(image_resized, axis=0)

      # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
      if float_input:
          input_data = (np.float32(input_data) - input_mean) / input_std

      # Perform the actual detection by running the model with the image as input
      interpreter.set_tensor(input_details[0]['index'],input_data)
      interpreter.invoke()

      # Retrieve detection results
      boxes = interpreter.get_tensor(output_details[1]['index'])[0] # Bounding box coordinates of detected objects
      classes = interpreter.get_tensor(output_details[3]['index'])[0] # Class index of detected objects
      scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

      detections = []

      # Loop over all detections and draw detection box if confidence is above minimum threshold
      for i in range(len(scores)):
          if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

              # Get bounding box coordinates and draw box
              # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
              ymin = int(max(1,(boxes[i][0] * imH)))
              xmin = int(max(1,(boxes[i][1] * imW)))
              ymax = int(min(imH,(boxes[i][2] * imH)))
              xmax = int(min(imW,(boxes[i][3] * imW)))
              
              cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 2)

              # Draw label
              object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
              label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
              labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2) # Get font size
              label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
              cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
              cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2) # Draw label text

              detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])

      
      # All the results have been drawn on the image, now display the image
      if txt_only == False: # "text_only" controls whether we want to display the image results or just save them in .txt files
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12,16))
        plt.imshow(image)
        plt.show()
      
      # Save detection results in .txt files (for calculating mAP)
      elif txt_only == True:

        # Get filenames and paths
        image_fn = os.path.basename(image_path)      
        base_fn, ext = os.path.splitext(image_fn)
        txt_result_fn = base_fn +'.txt'
        txt_savepath = os.path.join(savepath, txt_result_fn)

        # Write results to text file
        # (Using format defined by https://github.com/Cartucho/mAP, which will make it easy to calculate mAP)
        with open(txt_savepath,'w') as f:
            for detection in detections:
                f.write('%s %.4f %d %d %d %d\n' % (detection[0], detection[1], detection[2], detection[3], detection[4], detection[5]))

  return

### 3.2 Determine mAP Standard Model for test data
We now have a visual representation of how our model performs on test photos, but how can we quantify its accuracy? 


"Mean average precision" (mAP) is a prominent method for quantifying object detection model accuracy. Basically, the greater your model's mAP score, the better it is at recognizing objects in photos. Read this [Roboflow article](https://blog.roboflow.com/mean-average-precision/) to learn more about mAP.

To calculate our model's mAP score, we'll utilize the mAP calculator tool at https://github.com/Cartucho/mAP. To begin, we must clone the repository and delete any existing sample data. We'll also get a copy of the script I built for interacting with the calculator.


In [ ]:
%%bash
git clone https://github.com/Cartucho/mAP /content/mAP_test_std
cd /content/mAP_test_std
rm input/detection-results/* 
rm input/ground-truth/* 
rm input/images-optional/* 
wget https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/calculate_map.py

Cloning into '/content/mAP_test_std'...
--2023-06-10 09:47:06--  https://raw.githubusercontent.com/abriyanyusuf/C23PS423_ML/main/calculate_map.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4739 (4.6K) [text/plain]
Saving to: ‘calculate_map.py’

     0K ....                                                  100% 39.6M=0s

2023-06-10 09:47:06 (39.6 MB/s) - ‘calculate_map.py’ saved [4739/4739]



Next, we'll copy the images and annotation data from the **test** folder to the appropriate folders inside the cloned repository. These will be used as the "ground truth data" that our model's detection results will be compared to.


In [ ]:
!cp /content/images/test/* /content/mAP_test_std/input/images-optional # Copy images and xml files
!mv /content/mAP_test_std/input/images-optional/*.xml /content/mAP_test_std/input/ground-truth/  # Move xml files to the appropriate folder

The calculator tool expects annotation data in a format that's different from the Pascal VOC .xml file format we're using. Fortunately, it provides an easy script, `convert_gt_xml.py`, for converting to the expected .txt format.



In [ ]:
!python /content/mAP_test_std/scripts/extra/convert_gt_xml.py

Conversion completed!


In [ ]:
# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/images/test'   # Path to test images folder
PATH_TO_MODEL='/content/standard model/custom_model_lite/detect.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP_test_std/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(400, len(image_list)) # If there are more than 400 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

Starting inference on 299 images...
Finished inferencing!


Finally, let us compute mAP! The COCO metric for mAP @ 0.50:0.95 is a popular approach for reporting mAP. This implies that mAP is calculated at many IoU thresholds ranging from 0.50 to 0.95, and the results from each threshold are averaged to get a final mAP score. [Find out more here!](https://blog.roboflow.com/mean-average-precision/)

In [ ]:
%cd /content/mAP_test_std
!python calculate_map.py --labels=/content/labelmap.txt

/content/mAP_test_std
Calculating mAP at 0.50 IoU threshold...
100.00% = apple AP 
100.00% = banana AP 
100.00% = bread AP 
100.00% = bun AP 
99.33% = coin AP 
100.00% = doughnut AP 
100.00% = egg AP 
100.00% = fired_dough_twist AP 
100.00% = grape AP 
100.00% = lemon AP 
100.00% = litchi AP 
100.00% = mango AP 
100.00% = mooncake AP 
100.00% = orange AP 
100.00% = peach AP 
100.00% = pear AP 
100.00% = plum AP 
100.00% = qiwi AP 
100.00% = sachima AP 
100.00% = tomato AP 
mAP = 99.97%
Calculating mAP at 0.55 IoU threshold...
100.00% = apple AP 
100.00% = banana AP 
100.00% = bread AP 
100.00% = bun AP 
99.33% = coin AP 
100.00% = doughnut AP 
100.00% = egg AP 
100.00% = fired_dough_twist AP 
100.00% = grape AP 
100.00% = lemon AP 
100.00% = litchi AP 
100.00% = mango AP 
100.00% = mooncake AP 
100.00% = orange AP 
100.00% = peach AP 
100.00% = pear AP 
100.00% = plum AP 
100.00% = qiwi AP 
100.00% = sachima AP 
100.00% = tomato AP 
mAP = 99.97%
Calculating mAP at 0.60 IoU threshold...

The score reported at the end is your model's overall mAP score. Ideally, it should be above 50% (0.50). If it isn't, we can increase your model's accuracy by adding more images to your dataset. 

### 3.3 Determine mAP Quantized Model for test data
Now we have a visual sense of how our model performs on test images, but how can we quantitatively measure its accuracy? 

One popular methord for measuring object detection model accuracy is "mean average precision" (mAP). Basically, the higher the mAP score, the better your model is at detecting objects in images. To learn more about mAP, read through this [article from Roboflow](https://blog.roboflow.com/mean-average-precision/).

We'll use the mAP calculator tool at https://github.com/Cartucho/mAP to determine our model's mAP score. First, we need to clone the repository and remove its existing example data. We'll also download a script I wrote for interfacing with the calculator.

In [ ]:
%%bash
git clone https://github.com/Cartucho/mAP /content/mAP3
cd /content/mAP3
rm input/detection-results/* 
rm input/ground-truth/* 
rm input/images-optional/* 
wget https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py

Cloning into '/content/mAP3'...
--2023-06-10 11:36:54--  https://raw.githubusercontent.com/EdjeElectronics/TensorFlow-Lite-Object-Detection-on-Android-and-Raspberry-Pi/master/util_scripts/calculate_map_cartucho.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5397 (5.3K) [text/plain]
Saving to: ‘calculate_map_cartucho.py’

     0K .....                                                 100% 45.3M=0s

2023-06-10 11:36:54 (45.3 MB/s) - ‘calculate_map_cartucho.py’ saved [5397/5397]



Next, we'll copy the images and annotation data from the **test** folder to the appropriate folders inside the cloned repository. These will be used as the "ground truth data" that our model's detection results will be compared to.


In [ ]:
!cp /content/images/test/* /content/mAP3/input/images-optional # Copy images and xml files
!mv /content/mAP3/input/images-optional/*.xml /content/mAP3/input/ground-truth/  # Move xml files to the appropriate folder

The calculator tool expects annotation data in a format that's different from the Pascal VOC .xml file format we're using. Fortunately, it provides an easy script, `convert_gt_xml.py`, for converting to the expected .txt format.



In [ ]:
!python /content/mAP3/scripts/extra/convert_gt_xml.py

Conversion completed!


Okay, we've set up the ground truth data, but now we need actual detection results from our model. The detection results will be compared to the ground truth data to calculate the model's accuracy in mAP.

The inference function we defined in Step 7.1 can be used to generate detection data for all the images in the **test** folder. We'll use it the same as before, except this time we'll tell it to save detection results into the `detection-results` folder.

Click Play to run the following code block!

In [ ]:
# Set up variables for running inference, this time to get detection results saved as .txt files
PATH_TO_IMAGES='/content/images/test'   # Path to test images folder
PATH_TO_MODEL='/content/quant model/custom_model_lite/detect_quant.tflite'   # Path to .tflite model file
PATH_TO_LABELS='/content/labelmap.txt'   # Path to labelmap.txt file
PATH_TO_RESULTS='/content/mAP3/input/detection-results' # Folder to save detection results in
min_conf_threshold=0.1   # Confidence threshold

# Use all the images in the test folder
image_list = glob.glob(PATH_TO_IMAGES + '/*.jpg') + glob.glob(PATH_TO_IMAGES + '/*.JPG') + glob.glob(PATH_TO_IMAGES + '/*.png') + glob.glob(PATH_TO_IMAGES + '/*.bmp')
images_to_test = min(500, len(image_list)) # If there are more than 500 images in the folder, just use 500

# Tell function to just save results and not display images
txt_only = True

# Run inferencing function!
print('Starting inference on %d images...' % images_to_test)
tflite_detect_images(PATH_TO_MODEL, PATH_TO_IMAGES, PATH_TO_LABELS, min_conf_threshold, images_to_test, PATH_TO_RESULTS, txt_only)
print('Finished inferencing!')

Starting inference on 299 images...
Finished inferencing!


Finally, let's calculate mAP! One popular style for reporting mAP is the COCO metric for mAP @ 0.50:0.95. Basically, this means that mAP is calculated at several IoU thresholds between 0.50 and 0.95, and then the result from each threshold is averaged to get a final mAP score. [Learn more here!](https://blog.roboflow.com/mean-average-precision/)

I wrote a script to run the calculator tool at each IoU threshold, average the results, and report the final accuracy score. It reports mAP for each class and overall mAP. Click Play on the following two blocks to calculate mAP!

In [ ]:
%cd /content/mAP3
!python calculate_map_cartucho.py --labels=/content/labelmap.txt

/content/mAP3
Calculating mAP at 0.50 IoU threshold...
100.00% = apple AP 
90.91% = banana AP 
83.33% = bread AP 
100.00% = bun AP 
99.09% = coin AP 
89.47% = doughnut AP 
95.19% = egg AP 
80.00% = fired_dough_twist AP 
100.00% = grape AP 
40.48% = lemon AP 
80.00% = litchi AP 
40.74% = mango AP 
88.24% = mooncake AP 
27.78% = orange AP 
38.65% = peach AP 
68.01% = pear AP 
88.24% = plum AP 
61.46% = qiwi AP 
100.00% = sachima AP 
100.00% = tomato AP 
mAP = 78.58%
Calculating mAP at 0.55 IoU threshold...
100.00% = apple AP 
90.91% = banana AP 
83.33% = bread AP 
100.00% = bun AP 
99.09% = coin AP 
89.47% = doughnut AP 
95.19% = egg AP 
80.00% = fired_dough_twist AP 
100.00% = grape AP 
40.48% = lemon AP 
80.00% = litchi AP 
40.74% = mango AP 
88.24% = mooncake AP 
27.78% = orange AP 
38.65% = peach AP 
68.01% = pear AP 
88.24% = plum AP 
61.46% = qiwi AP 
100.00% = sachima AP 
100.00% = tomato AP 
mAP = 78.58%
Calculating mAP at 0.60 IoU threshold...
100.00% = apple AP 
90.91% = banana

The score reported at the end is your model's overall mAP score. Ideally, it should be above 50% (0.50). If it isn't, you can increase your model's accuracy by adding more images to your dataset. See my [dataset video](https://www.youtube.com/watch?v=v0ssiOY6cfg) for tips on how to capture good training images and improve accuracy.

Here are solutions to common errors that can occur while stepping through this notebook.

#If We Split Our Dataset Again We Should Archive Images Folder and Download it
We have to do this for troubleshooting if there is an error in the future

In [ ]:
%pwd

'/content'

In [ ]:
%cd /content
!zip -r images.zip images

Streaming output truncated to the last 5000 lines.
  adding: images/test/pear001T(14).xml (deflated 61%)
  adding: images/test/qiwi004T(5).JPG (deflated 1%)
  adding: images/test/mooncake004S(6).xml (deflated 62%)
  adding: images/test/apple017T(7).JPG (deflated 2%)
  adding: images/test/qiwi007T(12).JPG (deflated 1%)
  adding: images/test/fired_dough_twist005T(8).xml (deflated 61%)
  adding: images/test/plum001S(16).xml (deflated 61%)
  adding: images/test/apple016S(13).xml (deflated 61%)
  adding: images/test/orange007S(8).xml (deflated 62%)
  adding: images/test/apple002S(1).xml (deflated 61%)
  adding: images/test/fired_dough_twist007S(1).xml (deflated 62%)
  adding: images/test/orange005T(3).JPG (deflated 1%)
  adding: images/test/orange008T(11).JPG (deflated 2%)
  adding: images/test/fired_dough_twist002T(10).JPG (deflated 1%)
  adding: images/test/orange014S(1).xml (deflated 62%)
  adding: images/test/fired_dough_twist002T(4).xml (deflated 61%)
  adding: images/test/orange008T(1

In [ ]:
!cp '/content/images.zip' '/content/gdrive/MyDrive/Capstone Project Machine Learning' # Copy images.zip to our google drive